In [ ]:
# Everything except adaptive will probably work fine with other versions as well. 

# Python 3.6.6
import itertools
import random

# ipywidgets 7.4.1
from ipywidgets import interact_manual

# Numpy 1.15.1
import numpy as np

# Scipy 1.1.0
import scipy
import scipy.linalg
import scipy.spatial


# Kwant 1.3.3. May work with lower versions as well, I didn't test this
from kwant import builder, system, plotter
from kwant.linalg import lll
from kwant.builder import herm_conj, HermConjOfFunc
from kwant.lattice import TranslationalSymmetry
from kwant._common import get_parameters
from kwant.wraparound import *
import kwant.linalg
import kwant.lattice

# Adaptive 0.6 branch [111-no-flat-simplices], it doesn't work with on the master branch
import adaptive
adaptive.notebook_extension()


In [ ]:
from adaptive.learner.base_learner import uses_nth_neighbors
from adaptive.learner.learnerND import curvature_loss_function

In [ ]:
import plotly.io as pio
import plotly.graph_objs as go
import plotly
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [ ]:
import time

In [ ]:
do_plots = True

In [ ]:
# Define the lattice vectors of some common unit cells
# Hexagonal
a = -np.pi/6
unit_cell_vectors_hexagonal = [
    (        0,         1, 0), 
    (np.cos(a), np.sin(a), 0), 
    (        0,         0, 1)
]

# Simple cubic
unit_cell_vectors_simple_cubic = [
    (1, 0, 0), 
    (0, 1, 0), 
    (0, 0, 1)
]

# FCC
unit_cell_vectors_fcc = [
    (0,.5,.5), 
    (.5,.5,0), 
    (.5,0,.5)
]

# BCC
unit_cell_vectors_bcc = [
    (-.5, .5, .5), 
    ( .5,-.5, .5), 
    ( .5, .5,-.5)
]

In [ ]:
N = 15

In [ ]:
# Hexagonal
unit_cell_vectors = unit_cell_vectors_hexagonal
isolevel = -.5
crystal = 'Hexagonal'

In [ ]:
# Simple cubic
unit_cell_vectors = unit_cell_vectors_simple_cubic
isolevel = -.5

crystal = 'Simple cubic'

In [ ]:
# FCC
unit_cell_vectors = unit_cell_vectors_fcc
isolevel = -1

crystal = 'FCC'

In [ ]:
# BCC
unit_cell_vectors = unit_cell_vectors_bcc
isolevel = -0.5
# isolevel = -1
crystal = 'BCC'

In [ ]:
ax = dict(
    autorange=True,
    showgrid=False,
    zeroline=False,
    showline=False,
    ticks='',
    showticklabels=False,
    title=''
)

lo=go.Layout(
    scene = dict(
        xaxis = ax,
        yaxis = ax,
        zaxis = ax
    ),
    height=1500, width=1500
)

In [ ]:
time_start = time.time()

In [ ]:
# Choose some latice and atoms, like hexagonal in this case

unit_cell_atoms = [
    ( 0,  0,  0)
]

def create_syst():
    lat = kwant.lattice.Polyatomic(unit_cell_vectors, unit_cell_atoms, 'Some crystal')
   
    syst = kwant.Builder(kwant.TranslationalSymmetry(*lat.prim_vecs)) # infinite in all dimensions
    syst[lat.shape(lambda pos: True, (0, 0, 0))] = isolevel # onsite
    syst[lat.neighbors()] = -1 # hopping

    syst = kwant.wraparound.wraparound(syst).finalized()
    
    return syst

syst = create_syst()

def energies(params):
    H = syst.hamiltonian_submatrix(params=params)
    eigs = np.linalg.eigvalsh(H)
    return eigs

In [ ]:
# Do some math
B = np.asarray(unit_cell_vectors).T
A = np.linalg.pinv(B).T

neighbours = kwant.linalg.lll.voronoi(A)
lattice_points = np.concatenate(([[0,0,0]], neighbours))
lattice_points = 2 * np.pi * (lattice_points @ A.T)
vor = scipy.spatial.Voronoi(lattice_points)
brillouin_zone = vor.vertices[vor.regions[vor.point_region[0]]]
hull = scipy.spatial.ConvexHull(brillouin_zone)

bounds = list(map(tuple, np.vstack([
    np.min(brillouin_zone, axis=0),
    np.max(brillouin_zone, axis=0)
]).T * (1+1e-6))) # make bounds

In [ ]:
# convertion of coordinate system
def momentum_to_lattice(k):
    k, residuals = scipy.linalg.lstsq(A, k)[:2]
    if np.any(abs(residuals) > 1e-7):
        raise RuntimeError("Requested momentum doesn't correspond"
                           " to any lattice momentum.")
    return k

# Get the energy for a given k-vector
def E(k):
    k_x, k_y, k_z = momentum_to_lattice(k)
    return min(energies({'k_x': k_x, 'k_y': k_y, 'k_z': k_z}))

In [ ]:
x,y,z = np.meshgrid(np.linspace(*bounds[0],N), np.linspace(*bounds[1],N),np.linspace(*bounds[2],N))
X = list(zip(x.flat, y.flat, z.flat))
Y = [E(x) for x in X]

learnerH = adaptive.LearnerND(E, bounds=bounds)
lst = list(zip(X,Y))

np.random.shuffle(lst)

for x,y in lst:
    learnerH.tell(x, y)
    if learnerH.npoints %100==0:
        print(learnerH.npoints)

In [ ]:
normal_loss = curvature_loss_function(0.05)

@uses_nth_neighbors(1)
def loss(simplex, ys, nei, neiy):
    l = normal_loss(simplex, ys, nei, neiy)
    if min(ys) <= 0 and max(ys) >= 0:
        return l * 2
    return l

# Now do some adaptive, remember E = E(k)
learner = adaptive.LearnerND(E, hull, loss_per_simplex=loss)

hullH = scipy.spatial.ConvexHull(learnerH._bounds_points)
NpointsAdaptive = hull.volume / hullH.volume * N**3
print(NpointsAdaptive)

runner = adaptive.runner.simple(learner, goal=lambda l: l.npoints >= NpointsAdaptive) 


In [ ]:
if do_plots:
    learnerH.plot_isosurface(level=0)

In [ ]:
# plotly = ensure_plotly()

if do_plots:
    vertices, faces = learnerH._get_iso(0, which='surface')
    x, y, z = zip(*vertices)

    fig = plotly.figure_factory.create_trisurf(
        x=x, y=y, z=z, plot_edges=False,
        simplices=faces, title="")
    isosurface = fig.data[0]
    isosurface.update(lighting=dict(ambient=1, diffuse=1,
        roughness=1, specular=0, fresnel=0))

    fig.data = fig.data[:1]

    f = go.Figure(data=[fig.data[0], learner._get_hull_mesh()], layout=lo)
    pio.write_image(f, 'iso_homogeneous.png')
    plotly.offline.iplot(f)

In [ ]:
pts, triangles = learnerH._get_iso(level=0)
contained = learner._interior.find_simplex(pts, tol=1e-5)
count = 0
number_of_triangles_fully_container = 0
tri = []
x, y, z = zip(*pts)
for t in triangles:
    if any([contained[p] >= 0 for p in t]):
        count += 1
        tri.append(t)
    if all([contained[p] >= 0 for p in t]):
        number_of_triangles_fully_container += 1
        
        
if do_plots:
    fig = plotly.figure_factory.create_trisurf(
                x=x, y=y, z=z, plot_edges=False,
                simplices=tri, title="Isosurface")
    isosurface = fig.data[0]
    isosurface.update(lighting=dict(ambient=1, diffuse=1,
        roughness=1, specular=0, fresnel=0))
    
    f = go.Figure(data=[fig.data[0], learner._get_hull_mesh()], layout=lo)
    pio.write_image(f, 'iso_homogeneous.png')
    
    plot = plotly.offline.iplot(f)
print(count, number_of_triangles_fully_container)
number_of_triangles_homogeneous = count

In [ ]:
# plot = learner.plot_isosurface(level=0, hull_opacity=0.2)

In [ ]:
max(list(learner.data.values()))

In [ ]:
learner.npoints

In [ ]:
pts, triangles = learner._get_iso(level=0)
number_of_triangles_adaptive = len(triangles)
number_of_triangles_adaptive

In [ ]:
if do_plots:
    vertices, faces = learner._get_iso(0, which='surface')
    x, y, z = zip(*vertices)

    fig = plotly.figure_factory.create_trisurf(
        x=x, y=y, z=z, plot_edges=False,
        simplices=faces, title="")
    isosurface = fig.data[0]
    isosurface.update(lighting=dict(ambient=1, diffuse=1,
        roughness=1, specular=0, fresnel=0))

    fig.data = fig.data[:1]

    print(len(faces))
    f = go.Figure(data=[fig.data[0], learner._get_hull_mesh()], layout=lo)
    pio.write_image(f, 'iso_adaptive.png')
    plotly.offline.iplot(f)


In [ ]:
vertices, faces = learner._get_iso(0, which='surface')
x, y, z = zip(*vertices)
fig = plotly.figure_factory.create_trisurf(
    x=x, y=y, z=z, plot_edges=True,
    simplices=faces, title="")
isosurface = fig.data[0]
isosurface.update(lighting=dict(ambient=1, diffuse=1,
    roughness=1, specular=0, fresnel=0))

plotly.offline.iplot(fig)

In [ ]:
pts, triangles = learnerH._get_iso(level=0)
contained = learner._interior.find_simplex(pts, tol=1e-5)
count = 0
number_of_triangles_fully_container = 0
tri = []
x, y, z = zip(*pts)
for t in triangles:
    if any([contained[p] >= 0 for p in t]):
        count += 1
        tri.append(t)
    if all([contained[p] >= 0 for p in t]):
        number_of_triangles_fully_container += 1
        

        
vertices, faces = learnerH._get_iso(0, which='surface')
x, y, z = zip(*vertices)
fig = plotly.figure_factory.create_trisurf(
    x=x, y=y, z=z, plot_edges=True,
    simplices=tri, title="")
isosurface = fig.data[0]
isosurface.update(lighting=dict(ambient=1, diffuse=1,
    roughness=1, specular=0, fresnel=0))

plotly.offline.iplot(fig)
        


In [ ]:
vertices, faces = learnerH._get_iso(0, which='surface')
x, y, z = zip(*vertices)
fig = plotly.figure_factory.create_trisurf(
    x=x, y=y, z=z, plot_edges=True,
    simplices=faces, title="")
isosurface = fig.data[0]
isosurface.update(lighting=dict(ambient=1, diffuse=1,
    roughness=1, specular=0, fresnel=0))

plotly.offline.iplot(fig)

In [ ]:
sum((contained > 0))

In [ ]:
sum(learner._interior.find_simplex(learnerH.points) >= 0)

In [ ]:
hull = scipy.spatial.ConvexHull(learner.points)
hull.volume

In [ ]:
hullH = scipy.spatial.ConvexHull(learnerH.points)
hullH.volume

In [ ]:
# Expected number of points
hull.volume / hullH.volume * (N**3)

In [ ]:
scipy.spatial.ConvexHull(learner._interior.points).volume

In [ ]:
sum((learner._interior.find_simplex(learnerH.points, tol=1e-0) >= 0) * 1)

In [ ]:
sum((learner._interior.find_simplex(learnerH.points, tol=1e-1) >= 0) * 1)

In [ ]:
sum((learner._interior.find_simplex(learnerH.points, tol=1e-2) >= 0) * 1)

In [ ]:
sum((learner._interior.find_simplex(learnerH.points, tol=1e-3) >= 0) * 1)

In [ ]:
sum((learner._interior.find_simplex(learnerH.points, tol=1e-4) >= 0) * 1)

In [ ]:
sum((learner._interior.find_simplex(learnerH.points, tol=1e-5) >= 0) * 1)

In [ ]:
sum((learner._interior.find_simplex(learnerH.points, tol=1e-6) >= 0) * 1)

In [ ]:
s = learner._interior.find_simplex(learnerH.points, tol=1e-5)

In [ ]:
np.unique(s, return_counts=True)

In [ ]:
hull.volume / hullH.volume * 12000

In [ ]:
# Getallen die nodig zijn voor het verslag:


print(crystal)
print('Homo: N_tri', number_of_triangles_homogeneous)
print('      N_p  ', learnerH.npoints)
print('Adap: N_tri', number_of_triangles_adaptive)
print('      N_p  ', learner.npoints)
print('Ration', number_of_triangles_adaptive/number_of_triangles_homogeneous)

print('\n\nfully contained homo:', number_of_triangles_fully_container)
print(time.time() - time_start)


In [ ]:
x,y,z = learner.points.T

trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=5,
        color=z,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=1
    )
)

data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='3d-scatter-colorscale')